# Ensemble notebook
Takes the pickles made in the evaluation notebook to average probabilities together

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext tensorboard
#%matplotlib qt

In [2]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from collections import OrderedDict
import SimpleITK as sitk
#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
from collections import OrderedDict
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets

import pickle, subprocess
import scipy
import sklearn
import csv

import torchmetrics

#import initial_ml as iml
from gbm_project import data_prep as dp
from gbm_project.pytorch.run_model_torch import RunModel
from gbm_project.pytorch import resnet_spottune as rs
from MedicalNet.models import resnet

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
print(f"using {device} device")
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(True)

using cuda device


In [4]:
# all of the metrics used in the evaluation, can add or remove as desired
acc_fn = torchmetrics.classification.BinaryAccuracy(threshold=0.5)
auc_fn = torchmetrics.classification.BinaryAUROC()
spe_fn = torchmetrics.classification.BinarySpecificity()
pre_fn = torchmetrics.classification.BinaryPrecision()
sen_fn = torchmetrics.classification.BinaryRecall()
f1_fn = torchmetrics.classification.BinaryF1Score()
confusion = torchmetrics.classification.BinaryConfusionMatrix()
roc_fn = torchmetrics.classification.BinaryROC()

In [5]:
# resets metric functions, should be run for each successive evaluation of probabilities as values entered into functions are persistent until reset
acc_fn.reset()
auc_fn.reset()
spe_fn.reset()
sen_fn.reset()
pre_fn.reset()
f1_fn.reset()

In [6]:
#pull probability values from pkl files
# results_spottune is the location of the probability pickles
b_string_file_names = subprocess.check_output('ls -1 ./results_spottune')
file_name_list = b_string_file_names.decode().split('\n')
cwd = './results_spottune'

test_folds = {}
for name in file_name_list:
    if 'test' in name:
        with open(os.path.join(cwd,name), 'rb') as f:
            test_folds[name.replace('_test.pkl','')] = pickle.load(f)
            f.close()

In [7]:
# These lists form up the basis for various ensembles
# each list corresponds to an ensemble
dsc_mods = [
            'DSC_ap-rCBV',
            'DSC_PH',
            'DSC_PSR',
           ]
struct_mods = [
               'T2',
               'FLAIR',
               'T1',
               'T1GD',
              ]
dti_mods = [
            'DTI_AD',
            'DTI_FA',
            'DTI_RD',
            'DTI_TR',
           ]
dsc_struct = [
            'DSC_ap-rCBV',
            'DSC_PH',
            'DSC_PSR',
            'T2',
            'FLAIR',
            'T1',
            'T1GD',
             ]
dsc_dti = [
            'DSC_ap-rCBV',
            'DSC_PH',
            'DSC_PSR',
            'DTI_AD',
            'DTI_FA',
            'DTI_RD',
            'DTI_TR',
          ]
all_mods = [
            'DSC_ap-rCBV',
            'DSC_PH',
            'DSC_PSR',
            'DTI_AD',
            'DTI_FA',
            'DTI_RD',
            'DTI_TR',
            'T2',
            'FLAIR',
            'T1',
            'T1GD',
          ]
dti_struct = [
            'DTI_AD',
            'DTI_FA',
            'DTI_RD',
            'DTI_TR',
            'T2',
            'FLAIR',
            'T1',
            'T1GD',
          ]

# The name of the model to make ensembles of, refers to a specific training
trainings = [
             'M_metric',
            ]

training_sets = {}

# sets up the models that go into each ensemble
# i.e. DSC_DTI will grab the models corresponding to all of the DSC and DTI derivatives
for t in trainings:
    training_sets[f"spottune_DSC_{t}"] = [f"spottune_{m}_{t}" for m in dsc_mods]
    training_sets[f"spottune_struct_{t}"] = [f"spottune_{m}_{t}" for m in struct_mods]
    training_sets[f"spottune_DTI_{t}"] = [f"spottune_{m}_{t}" for m in dti_mods]
    training_sets[f"spottune_all_{t}"] = [f"spottune_{m}_{t}_comb_all" for m in all_mods]
    training_sets[f"spottune_DSC_DTI_{t}"] = [f"spottune_{m}_{t}_comb_dti" for m in dsc_dti]
    training_sets[f"spottune_DSC_struct_{t}"] = [f"spottune_{m}_{t}_comb_struct" for m in dsc_struct]
    training_sets[f"spottune_DTI_struct_{t}"] = [f"spottune_{m}_{t}_comb_dti_struct" for m in dti_struct]
    #training_sets[f"transfer_DSC_{t}"] = [f"transfer_{m}_{t}" for m in dsc_mods]
    #training_sets[f"transfer_DTI_{t}"] = [f"transfer_{m}_{t}" for m in dti_mods]
    #training_sets[f"transfer_struct_{t}"] = [f"transfer_{m}_{t}" for m in struct_mods]
    #training_sets[f"transfer_all_{t}"] = [f"transfer_{m}_{t}_comb_all" for m in all_mods]
    #training_sets[f"transfer_DSC_DTI_{t}"] = [f"transfer_{m}_{t}_comb_dti" for m in dsc_dti]
    #training_sets[f"transfer_DSC_struct_{t}"] = [f"transfer_{m}_{t}_comb_struct" for m in dsc_struct]
    #training_sets[f"transfer_DTI_struct_{t}"] = [f"transfer_{m}_{t}_comb_dti_struct" for m in dti_struct]
    #training_sets[f"random_DSC_{t}"] = [f"random_{m}_{t}" for m in dsc_mods]
    #training_sets[f"random_DTI_{t}"] = [f"random_{m}_{t}" for m in dti_mods]
    #training_sets[f"random_struct_{t}"] = [f"random_{m}_{t}" for m in struct_mods]
    #training_sets[f"random_all_{t}"] = [f"random_{m}_{t}_comb_all" for m in all_mods]
    #training_sets[f"random_DSC_DTI_{t}"] = [f"random_{m}_{t}_comb_dti" for m in dsc_dti]
    #training_sets[f"random_DSC_struct_{t}"] = [f"random_{m}_{t}_comb_struct" for m in dsc_struct]
    #training_sets[f"random_DTI_struct_{t}"] = [f"random_{m}_{t}_comb_dti_struct" for m in dti_struct]

In [8]:
# since the DTI and struct modalitites have more patients, these lines grab only those patients that are common to the DSC modalities in the testing dataset, dropping extra patients
# the 'spottune_...' strings reference a placeholder model for each modality, replace as necessary with your own model
dsc_pats = test_folds['spottune_DSC_PH_M_metric']['fold_0'][0]
dti_pats = test_folds['spottune_DTI_AD_M_metric']['fold_0'][0]
struct_pats = test_folds['spottune_T2_M_metric']['fold_0'][0]

# the locations of the common patients
dti_locs = [dti_pats.get_loc(pat) for pat in dsc_pats]
struct_locs = [struct_pats.get_loc(pat) for pat in dsc_pats]

In [9]:
# Calculate the average probabilities for each fold for the testing dataset
# These are averaging the 100 GS samplings into one set of probabilities for each fold

test_fold_averages = {}
fold_list = ['fold_0',
             'fold_1',
             'fold_2',
             'fold_3',
             'fold_4'
            ]
for training in test_folds:
    running_avg = []
    dsc_dti_running_avg = []
    dsc_struct_running_avg = []
    dti_struct_running_avg = []
    val_running_avg = []
    for i, fold in enumerate(fold_list):
        if fold not in test_folds[training]:
            print(f"skipping {fold} for training: {training}")
            continue
        running_avg.append(np.average(test_folds[training][f"{fold}"][1][:,0], axis=0))
        if 'DTI' in training:
            dsc_dti_running_avg.append(np.average(test_folds[training][f"{fold}"][1][:,0][:, dti_locs], axis=0))
            dti_struct_running_avg.append(np.average(test_folds[training][f"{fold}"][1][:,0][:, dti_locs], axis=0))
        if np.any([t in training for t in ['_T2_', '_FLAIR_', '_T1_', '_T1GD_']]):
            if 'DSC' in training: continue
            dsc_struct_running_avg.append(np.average(test_folds[training][f"{fold}"][1][:,0][:, struct_locs], axis=0))
            dti_struct_running_avg.append(np.average(test_folds[training][f"{fold}"][1][:,0][:, struct_locs], axis=0))

    # averaging probabilities between each fold
    test_fold_averages[training] = np.mean(running_avg, axis=0)
    if 'DSC' in training:
        test_fold_averages[f"{training}_comb_dti"] = np.mean(running_avg, axis=0) 
        test_fold_averages[f"{training}_comb_struct"] = np.mean(running_avg, axis=0) 
        test_fold_averages[f"{training}_comb_all"] = np.mean(running_avg, axis=0) 
    if np.any(dsc_dti_running_avg): 
        test_fold_averages[f"{training}_comb_dti"] = np.mean(dsc_dti_running_avg, axis=0)
        test_fold_averages[f"{training}_comb_all"] = np.mean(dsc_dti_running_avg, axis=0)
    if np.any(dsc_struct_running_avg): 
        test_fold_averages[f"{training}_comb_struct"] = np.mean(dsc_struct_running_avg, axis=0)
        test_fold_averages[f"{training}_comb_all"] = np.mean(dsc_struct_running_avg, axis=0)
    if np.any(dti_struct_running_avg): 
        test_fold_averages[f"{training}_comb_dti_struct"] = np.mean(dti_struct_running_avg, axis=0)
    val_fold_averages[training] = val_running_avg

# averaging probabilities between modalities
for s in training_sets.keys():
    test_mean_probs = [test_fold_averages[t] for t in training_sets[s]]
    test_fold_averages[f"{s}"] = np.mean(test_mean_probs, axis=0)

In [10]:
#removes extraneous items that were used for intermediate combination steps
test_fold_averages = {k:v for k,v in test_fold_averages.items() if 'comb' not in k}

In [ ]:
# calculation of performance metrics
# saved to 'test_results...' file

metrics_avg = {}
for training in test_fold_averages:
    target = []
    print(training)
    # get the target values for the testing dataset
    try:
        target = test_folds[training]['fold_0'][1][:,1][0]
    except:
        # the targets are all the same, but requires a different string to pull the target from the correct location depending on the ensemble name
        if 'DSC_DTI' in training:
            target = test_folds[training.replace('DSC_DTI', 'DSC_PH')]['fold_0'][1][:,1][0]
        elif 'DSC_struct' in training:
            target = test_folds[training.replace('DSC_struct', 'DSC_PH')]['fold_0'][1][:,1][0] 
        elif 'DTI_struct' in training:
            target = test_folds[training.replace('DTI_struct', 'DSC_PH')]['fold_0'][1][:,1][0] 
        elif 'DSC_splits' in training:
            target = test_folds[training.replace('DSC_splits', 'DSC_PH_plateau_0')]['fold_0'][1][:,1][0] 
        elif 'DSC' in training:
            target = test_folds[training.replace('DSC','DSC_PH')]['fold_0'][1][:,1][0]
        elif 'struct' in training:
            target = test_folds[training.replace('struct','T2')]['fold_0'][1][:,1][0]
        elif 'DTI' in training:
            target = test_folds[training.replace('DTI','DTI_AD')]['fold_0'][1][:,1][0]
        elif 'all' in training:
            target = test_folds[training.replace('all', 'DSC_PH')]['fold_0'][1][:,1][0] 
            
        print('changing target for combined modalities')

    # calculates the different selected metrics
    acc = acc_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    auc = auc_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    spe = spe_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    pre = pre_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    sen = sen_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    f1 = f1_fn(torch.tensor(test_fold_averages[training]), torch.tensor(target))
    metrics_avg[training] = (acc, auc, sen, spe, pre, f1)
with open('test_results_09132023.csv', 'w', newline='') as f:
    w = csv.writer(f)
    f.write('model, ACC, AUC, SEN, SPE, PRE, F1')
    f.write('\r\n')
    for key, val in metrics_avg.items():
        w.writerow([key] + [str(v) for v in val])
    #w.writerows(metrics_avg.items())
    f.close()